In [ ]:
import data_call
import numpy as np
d_name='Muraro'
X,label=data_call.get_dataset(d_name, mode='PCA')
# X_cg,label_new=data_call.get_dataset(d_name, mode='CG')
#
# if type(X_cg) is not np.ndarray:
#     X_cg=X_cg.toarray()
#
#
# print(X.shape,label.shape)
# print(type(X),type(X_cg))
# print(X_cg.shape,label_new.shape)
#
#
# hmap={}
# t=0
# for i in sorted(set(label)):
#     hmap[i]=t
#     t+=1
# new_label= [hmap[i] for i in label]
# y= new_label
# true_k=len(set(y))

In [ ]:
import scanpy as sc
nk_path='/Users/chandrasekharmukherjee/PycharmProjects/Personal/cplearn_biotest/NK_cells.h5ad'
adata = sc.read_h5ad(nk_path)
X=adata.obsm['X_HARMONY']
label=adata.obs['ident'].to_numpy()

In [1]:
f_path='/Users/chandrasekharmukherjee/Data/scRNA_IBD/'
d_path=['bcells_final.h5ad',
'cd4tcells_final.h5ad',
'cd8tcells_final.h5ad',
'epicolonic_final.h5ad',
'fibperi_final.h5ad',
'ilc_final.h5ad',
'ilealepi_final.h5ad',
'myeloid_final.h5ad',
'plasmacells_final.h5ad']

subset_number=1

import scanpy as sc
adata=sc.read_h5ad(f_path+d_path[subset_number])
X=adata.obsm['X_harmony']
n=X.shape[0]
print(n)
ob_label=adata.obs['final_analysis'].to_numpy()
pt_label=adata.obs['Patient'].to_numpy()


145704


In [ ]:
from cplearn_v2.corespect import Corespect
corespect = Corespect(X)

In [8]:
print(X.shape)

(145704, 50)


In [ ]:
Total_Layers=[]
for purify in [1]:
    corespect.extract_layers(method='stable_rank',core_fraction=0.2,layer_extraction_params={'q':20,'r':10,'purify':purify})
    Layers=corespect.layers_
    Total_Layers.append(Layers)
    c=0
    for layer in Layers:
        c+=len(layer)
        print(len(layer),len(set(layer)))
    print(c,X.shape[0])

    print('\n\n')

In [ ]:
corespect.extract_layers()
Layers_part=corespect.layers_
c=0
for layer in Layers_part:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

In [ ]:
corespect.extract_layers(method='structural_majority')
Layers_struc=corespect.layers_
c=0
for layer in Layers_struc:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

In [ ]:
corespect.extract_layers(method='stable_rank',core_fraction=0.2,layer_extraction_params={'q':20,'r':10})
Layers=corespect.layers_
c=0
for layer in Layers:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

#In Stable_Rank we should also clean up the core by using the clean recursive majority.
#Points that are


In [ ]:
corespect.extract_layers(method='stable_rank',core_fraction=0.2,layer_extraction_params={'q':20,'r':10,'purify':1})
Pure_Layers_1=corespect.layers_
c=0
for layer in Pure_Layers_1:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

In [ ]:
corespect.extract_layers(method='stable_rank',core_fraction=0.2,layer_extraction_params={'q':20,'r':10,'purify':2})
Pure_Layers_2=corespect.layers_
c=0
for layer in Pure_Layers_2:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

In [ ]:
import numpy as np
import plotly.express as px
def plot_embedding(embedding, labels=None):
    """
    Plot a 2D embedding with optional labels for coloring.

    Args:
        embedding (ndarray): shape (n_samples, 2)
        labels (array-like, optional): labels for coloring points
    """
    if labels is None:
        labels = np.zeros(embedding.shape[0])  # default: all same

    fig = px.scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        color=labels.astype(str),  # convert to string for discrete legend
        title="2D Embedding Visualization",
        labels={"x": "Dim 1", "y": "Dim 2", "color": "Label"}
    )

    fig.update_layout(
        width=1000,
        height=800,
        title="Custom size Plotly figure"
    )


    fig.show()


In [ ]:
import umap


reducer=umap.UMAP()
X_umap=reducer.fit_transform(X)

In [ ]:
plot_embedding(X_umap)

In [ ]:
plot_embedding(X_umap,labels=ob_label)

In [ ]:
from cplearn_v2.corespect.ranking import FlowRank
layer=Total_Layers[1][0]
n=X.shape[0]
hmap=-1*np.ones(n)
for i,node in  enumerate(layer):
    hmap[node]=i

X_c=X[np.array(layer).astype(int)]
final_score=FlowRank(X_c,{'q':10,'r':5})
sorted_nodes= sorted(final_score, key=lambda k: final_score[k], reverse=True)
core_nodes_c=np.array(sorted_nodes[:int(0.3*len(layer))]).astype(int)
new_cores=[layer[ell] for ell in core_nodes_c]



In [ ]:
final_score=FlowRank(X,{'q':20,'r':10})
sorted_nodes= sorted(final_score, key=lambda k: final_score[k], reverse=True)
core_nodes=np.array(sorted_nodes[:int(0.5*n)]).astype(int)

In [ ]:
X_c_umap=reducer.fit_transform(X_c)

In [ ]:
r_label=-1*np.ones(X_c.shape[0])
for ell in core_nodes_c:
    r_label[ell]=0

plot_embedding(X_c_umap,labels=r_label)

In [ ]:
round_label=-1*np.ones(X.shape[0])

for node in layer:
    round_label[node]=0

for node in new_cores:
    round_label[node]=1

plot_embedding(X_umap,labels=round_label)

round_label=-1*np.ones(X.shape[0])


for node in core_nodes:
    round_label[node]=0


plot_embedding(X_umap,labels=round_label)

In [ ]:
round_label=-1*np.ones(X.shape[0])
for i,layer in enumerate(Layers_part):
    for node in layer:
        round_label[node]=i

plot_embedding(X_umap,labels=round_label)

In [ ]:
plot_embedding(X_umap,labels=ob_label)

In [ ]:
plot_embedding(X_umap[Total_Layers[0][0]],labels=ob_label[Total_Layers[0][0]])

In [ ]:
from cplearn_v2.corespect.ranking import FlowRank
layer=Total_Layers[0][0]
n=X.shape[0]
hmap=-1*np.ones(n)
for i,node in  enumerate(layer):
    hmap[node]=i

X_c=X[np.array(layer).astype(int)]
final_score=FlowRank(X_c,{'q':10,'r':5})


In [ ]:
Layers_h=Total_Layers[0]
layer_h=Layers_h[0]
sorted_nodes= sorted(final_score, key=lambda k: final_score[k], reverse=True)
core_nodes_c=np.array(sorted_nodes[:int(0.2*len(layer))]).astype(int)
new_cores=[layer_h[ell] for ell in core_nodes_c]

In [ ]:
round_here=-1*np.zeros(X.shape[0])

for i,layer in enumerate(Layers_h):
    for node in layer:
        round_here[node]=i+1

for node in new_cores:
    round_here[node]=0


plot_embedding(X_umap,labels=round_here)


In [ ]:
import numpy as np
for Layers_ch in Total_Layers:
    round_label=-1*np.ones(X.shape[0])
    for i,layer in enumerate(Layers_ch):
        for node in layer:
            round_label[node]=i

    plot_embedding(X_umap,labels=round_label)
    print("Next")

In [ ]:
import numpy as np
for Layers_ch in [Pure_Layers]:
    round_label=-1*np.ones(X.shape[0])
    for i,layer in enumerate(Layers_ch):
        for node in layer:
            round_label[node]=i

    plot_embedding(X_umap,labels=round_label)
    print("Next")

In [ ]:
from cplearn_v2.utils.clustering_algo import louvain
from cplearn_v2.utils.gen_utils import get_kNN

from sklearn.metrics import normalized_mutual_info_score as NMI
import networkx as nx



G = nx.DiGraph()
n=X.shape[0]

for i in range(n):
    G.add_node(i)

knn_list, _ = get_kNN(X, q=20)
for i in range(n):
    for j in knn_list[i, :]:
        if i != j:
            G.add_edge(i, j, weight=1)



In [ ]:
label_list=[]
for _ in range(5):

    label_map=louvain(G)
    labels=-1*np.ones(n)
    for i in range(n):
        labels[i]=label_map[i]

    label_list.append(labels)


In [ ]:
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import adjusted_mutual_info_score as ARI

def calc_run_wise_similarity(cluster_list_f):
    cluster_list_f = sorted(cluster_list_f, key=lambda x: len(set(x)))

    for clusters_f in cluster_list_f:
        print(len(set(clusters_f)), end=', ')

    print()

    print("NMI Matrix:\n")
    nmi_mat = np.zeros((len(cluster_list_f), len(cluster_list_f)))
    for i in range(len(cluster_list_f)):
        for j in range(len(cluster_list_f)):
            nmi_mat[i, j] = NMI(cluster_list_f[i], cluster_list_f[j])

    for row in nmi_mat:
        print(" ".join(f"{val:6.2f}" for val in row))

    print("\n\n")

    print("ARI Matrix:\n")
    ari_mat = np.zeros((len(cluster_list_f), len(cluster_list_f)))
    for i in range(len(cluster_list_f)):
        for j in range(len(cluster_list_f)):
            ari_mat[i, j] = ARI(cluster_list_f[i], cluster_list_f[j])

    for row in ari_mat:
        print(" ".join(f"{val:6.2f}" for val in row))

In [ ]:
import networkx as nx
import igraph as ig
from cplearn_v2.utils.gen_utils import get_kNN

In [ ]:
knn_list, _ = get_kNN(X, q=20)

In [ ]:
G1=nx.DiGraph()
for i in range(n):
    for j in knn_list[i, :]:
        if i != j:
            G1.add_edge(i, j, weight=1)

In [ ]:
from numba import njit

@njit
def get_edge_list(knn_list_local):
    edge_list=[]
    for i_loc in range(n):
        for j_loc in knn_list_local[i_loc, :]:
            if i_loc != j_loc:
                edge_list.append((i_loc,j_loc))

    return edge_list

In [ ]:
e_list=get_edge_list(knn_list)

In [ ]:
G2=ig.Graph(directed=True)
G2.add_vertices(n)

In [ ]:
G2.add_edges(e_list)

In [ ]:
from cplearn_v2.utils.clustering_algo import louvain

In [ ]:
label_map=louvain(G1)

In [ ]:
pip install leidenalg

In [ ]:
import leidenalg
partition = leidenalg.find_partition(
    G2,
    leidenalg.ModularityVertexPartition
)

In [ ]:
partition_n = leidenalg.find_partition(
    G2,
    leidenalg.RBConfigurationVertexPartition,  # modularity-based partition
    resolution_parameter=1                   # <-- control resolution here
)

In [ ]:
for clusters in partition_n:
    print(len(clusters),end=' ')

In [ ]:
label_new=-1*np.ones(n)
for i,clusters in enumerate(partition_n):
    for node in clusters:
        label_new[node]=i

In [ ]:
label_louv=-1*np.ones(n)
for i in range(n):
    label_louv[i]=label_map[i]

print(len(set(label_louv)))

In [ ]:
from sklearn.metrics import adjusted_mutual_info_score as AMI
from sklearn.metrics import adjusted_rand_score as ARI

print(ARI(label_louv,label_new),ARI(label_new,label_louv))

In [87]:

print(min(deg_list),max(deg_list))

20 20


In [ ]:
edge_list_again=G2.get_edgelist()

In [ ]:
@njit
def get_induced_edge_list(edge_list,subset,n):

    hmap=-1*np.ones(n)
    for ell in subset:
        hmap[ell]=1

    new_edge_list=[]

    for u,v in edge_list:
        if hmap[u]==1 and hmap[v]==1:
            new_edge_list.append((u,v))

    return new_edge_list

In [ ]:
new_edge_list=get_induced_edge_list(edge_list_again,new_cores,n)

In [ ]:
# Store IDs as a vertex attribute
G2.vs["id"] = list(range(G2.vcount()))
subg = G2.induced_subgraph(new_cores)

In [ ]:
print(G2.clique_number())

In [ ]:
print(G1.number_of_nodes())

In [ ]:
from numba.typed import List

ng_list=[]

#Preparing for numba. ng_list is a n*r matrix.
for ell in range(n):
  x=[]
  for v in G1.neighbors(ell):
    x.append(int(v))

  ng_list.append(List(x))

In [ ]:
print(n)

In [ ]:
from numba.typed import List

@njit
def get_numba_elist(edge_list,n):

    #Preparing for numba. ng_list is a n*r matrix.
    ng_list = List()
    for _ in range(n):
        ng_list.append(List.empty_list(np.int64))

    for u,v in edge_list:
        ng_list[u].append(v)

    # f_ng_list=[]
    # for ell in range(n):
    #       f_ng_list.append(List(ng_list[ell]))

@njit
def build_neighbors(edge_list, n, max_deg):
    neigh = -np.ones((n, max_deg), dtype=np.int64)  # -1 = empty slot
    deg = np.zeros(n, dtype=np.int64)

    for u,v in edge_list:
        idx = deg[u]
        neigh[u, idx] = v
        deg[u] += 1
    return neigh, deg


build_neighbors(e_list,n,20)

#get_numba_elist(e_list,n)


In [ ]:
build_neighbors(e_list,n,20)

In [69]:
from cplearn_v2.corespect import ranking as ranking_old
final_score=ranking_old.FlowRank(X,{'q':20,'r':20})

In [58]:
from cplearn_v3.corespect import ranking
import importlib
importlib.reload(ranking)

<module 'cplearn_v3.corespect.ranking' from '/Users/chandrasekharmukherjee/PycharmProjects/cplearn/cplearn_v3/corespect/ranking.py'>

In [53]:
final_score_new_1=ranking.FlowRank(X,20)

(145704, 20)
numba cp1
numba cp2


In [63]:
knn_list,_=get_kNN(X,20)

In [64]:
print(type(knn_list))

<class 'numpy.ndarray'>


In [65]:
final_score_new=ranking.FlowRank(knn_list,20)

20
(145704, 20)
numba cp1
numba cp2


In [68]:
print(max(final_score.values()),min(final_score))

0.9999999593730639 0


In [26]:
print(type(final_score_new))

<class 'dict'>


In [71]:
sorted_nodes= sorted(final_score, key=lambda k: final_score[k], reverse=True)
sorted_nodes_new= sorted(final_score, key=lambda k: final_score_new[k], reverse=True)

In [72]:
for ell in range(1,10,1):
    core_nodes=np.array(sorted_nodes[:int(ell*n/10)]).astype(int)
    core_nodes_new=np.array(sorted_nodes_new[:int(ell*n/10)]).astype(int)

    print(f"{(len(set(core_nodes).intersection(set(core_nodes_new))))/len(core_nodes):.3f}")


0.993
0.996
0.997
0.997
0.998
0.998
0.999
0.998
0.999


In [31]:
print(type(knn_list))

<class 'numpy.ndarray'>


In [79]:
from cplearn_v3.utils.gen_utils import get_edge_list
import igraph as ig

e_list = get_edge_list(knn_list,X.shape[0])
G = ig.Graph(directed=True)
G.add_vertices(X.shape[0])
G.add_edges(e_list)
G.vs["id"] = list(range(G.vcount()))

In [84]:
from cplearn_v3.corespect import layer_extraction
import importlib
importlib.reload(layer_extraction_v1)

core_nodes_new=np.array(sorted_nodes_new[:int(0.5*n)]).astype(int)
labels=layer_extraction_v1.cluster_subset(G,core_nodes_new)




Step 1
Updated
